# Sieci konwolucyjne

autor: Paulina Tomaszewska

Tak jak na poprzednich zajęciach będziemy klasyfikować obrazki z ręcznie napisanymi cyframi ze zbioru MNIST. Tym razem zastosujemy jednak inny rodzaj sieci neuronowych - sieci konwolucyjne.
________________________________________________________________________________

Kod poniżej ma bardzo podobną strukturę do tego na zajęciach o perceptronie wielowarstwowym (MLP). W tym przypdku wydrębiono poszczególne fragmenty kodu do funkcji.
________________________________________________________________________________

Kod aktualnie nie wykonuje się poprawnie - pojawiają się komunikaty błędu. Spróbuj je poprawić.

W kodzie zakomentowane są linijki kodu dot. warstwy Dropout. Odkomentuj je i przeanalizuj jak to wpływa na trening w tym zmiana wartości parametru tej warstwy.
W przypadku problemów z tym zadaniem, zadaj pytanie na Discord.

### Import pakietów

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

### Ustawienie ziarna losowości

In [2]:
import numpy as np
import random

# Ustawiamy ziarno losowości, aby zapewność reprodukowalność (powtarzalność wyników)
# Ziarno losowości jest kluczowe np. podczas inicjalizacji wag
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

### Przygotowanie danych

In [ ]:
batch_size = 64

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

full_train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Definicja modelu - sieć konwolucyjna

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=3)
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
        #self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x= self.relu(self.conv2(x))
        x = self.pool(x)
        x= self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1) # jest to równoważne operacji spłaszczenia (ang. flatten)
        x = self.relu(self.fc1(x))
        #x = self.dropout(x)
        x = self.fc2(x)
        return x

### Funkcje do trenowania i ewaluacji modelu

In [5]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for data, target in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

def evaluate(model, data_loader, criterion, device):
    model.eval()
    data_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            outputs = model(data)
            data_loss += criterion(outputs, target).item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == target).sum().item()
    data_loss /= len(data_loader)
    accuracy = correct / len(data_loader.dataset)
    return data_loss, accuracy

### Pętla treningowa
W tym przypadku, zrezygnowano z mechanizmu Early stopping, aby kod był krótszy i łatwiejszy do przeanalizowania w celu znalezienia i poprawy błędów.

In [6]:
max_epochs = 5
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
train_losses = []
val_losses = []
val_accuracies = []
train_accuracies = []

for epoch in range(max_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    _, train_accuracy = evaluate(model, train_loader, criterion, device)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}/{max_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

### Finalna ewaluacja modelu na zbiorze testowym

In [ ]:
test_loss_final, test_accuracy_final = evaluate(model, test_loader, criterion, device)
print(f"\nFinal Test Loss: {test_loss_final:.4f}, Final Test Accuracy: {test_accuracy_final:.4f}")

### Wykresy straty i dokładności (loss and accuracy)

Kod do stworzenia każdego z dwóch wykresów jest bardzo podobny. Spróbuj napisać funkcję pomocniczą, aby uniknąć powtarzających się linii kodu.

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label='Validation Accuracy')
plt.plot(train_accuracies, label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

#Podsumowanie modelu
Można wygenerować w sposób automatyczny podsumowanie tego jakie są warstwy modelu.

In [ ]:
from torchsummary import summary
summary(model, (1, 28, 28)) # podajemy obiekt modelu i rozmiar jednej próbki z danych wejściowych do modelu

Przenanlizuj pola dotyczące liczby parametrów (wag) w konkretnych warstwach, a także całościowe podsumowanie dla całej sieci. Zauważ, że zgodnie z oczekiwaniami w przypadku warstwy pooling nie ma wag.

Zastanów się kiedy w polu 'Non-trainable params' wartość będzie niezerowa.

Podpowiedź: transfer learning